In [1]:
stocks = {
    "A": {"length": 5, "cost": 6},
    "B": {"length": 6, "cost": 7},
    "C": {"length": 9, "cost": 10},
}

finish = {
    "S": {"length": 2, "demand": 20},
    "M": {"length": 3, "demand": 10},
    "L": {"length": 4, "demand": 20},
}

In [2]:
SOLVER_MILO = "highs"
SOLVER_MINLO = "ipopt"

In [3]:
def make_naive_patterns(stocks, finish):
    """
    Generates patterns of feasible cuts from stock lengths to meet specified finish lengths.

    Parameters:
    stocks (dict): A dictionary where keys are stock identifiers and values are dictionaries
                   with key 'length' representing the length of each stock.

    finish (dict): A dictionary where keys are finish identifiers and values are dictionaries
                   with key 'length' representing the required finish lengths.

    Returns:
    patterns (list): A list of dictionaries, where each dictionary represents a pattern of cuts.
                   Each pattern dictionary contains 'stock' (the stock identifier) and 'cuts'
                   (a dictionary where keys are finish identifiers and the value is the number
                   of cuts from the stock for each finish).
    """

    patterns = []
    for f in finish:
        feasible = False
        for s in stocks:
            # max number of f that fit on s
            num_cuts = int(stocks[s]["length"] / finish[f]["length"])

            # make pattern and add to list of patterns
            if num_cuts > 0:
                feasible = True
                cuts_dict = {key: 0 for key in finish.keys()}
                cuts_dict[f] = num_cuts
                patterns.append({"stock": s, "cuts": cuts_dict})

        if not feasible:
            print(f"No feasible pattern was found for {f}")
            return []

    return patterns

patterns = make_naive_patterns(stocks, finish)
display(patterns)

[{'stock': 'A', 'cuts': {'S': 2, 'M': 0, 'L': 0}},
 {'stock': 'B', 'cuts': {'S': 3, 'M': 0, 'L': 0}},
 {'stock': 'C', 'cuts': {'S': 4, 'M': 0, 'L': 0}},
 {'stock': 'A', 'cuts': {'S': 0, 'M': 1, 'L': 0}},
 {'stock': 'B', 'cuts': {'S': 0, 'M': 2, 'L': 0}},
 {'stock': 'C', 'cuts': {'S': 0, 'M': 3, 'L': 0}},
 {'stock': 'A', 'cuts': {'S': 0, 'M': 0, 'L': 1}},
 {'stock': 'B', 'cuts': {'S': 0, 'M': 0, 'L': 1}},
 {'stock': 'C', 'cuts': {'S': 0, 'M': 0, 'L': 2}}]

In [4]:
from ortools.linear_solver import pywraplp

def cut_patterns_ortools(stocks, finish, patterns):
    # Initialize the solver
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        raise Exception('Solver not created.')

    # Define sets
    F = list(finish.keys())
    P = list(range(len(patterns)))

    # Parameters
    c = [stocks[patterns[p]["stock"]]["cost"] for p in P]
    a = {(f, p): patterns[p]["cuts"].get(f, 0) for p in P for f in F}
    demand_finish = {f: finish[f]["demand"] for f in F}

    # Variables
    x = [solver.IntVar(0, solver.infinity(), f'x[{p}]') for p in P]

    # Objective function: minimize stock used
    objective = solver.Objective()
    for p in P:
        objective.SetCoefficient(x[p], c[p])
    objective.SetMinimization()

    # Constraints
    for f in F:
        constraint = solver.Constraint(demand_finish[f], solver.infinity())
        for p in P:
            constraint.SetCoefficient(x[p], a[f, p])

    # Solve the problem
    status = solver.Solve()

    if status != pywraplp.Solver.OPTIMAL:
        raise Exception('The problem does not have an optimal solution.')

    # Get the results
    x_values = [x[p].solution_value() for p in P]
    cost = solver.Objective().Value()

    return x_values, cost

# Example usage:
x, cost = cut_patterns_ortools(stocks, finish, patterns)
print(f"Optimal pattern choice: {x}")
print(f"Minimum cost: {cost}")

Optimal pattern choice: [1.0, 6.0, 0.0, 0.0, 2.0, 2.0, 0.0, 0.0, 10.0]
Minimum cost: 182.0


In [5]:
from pulp import LpMaximize, LpMinimize, LpProblem, LpVariable, lpSum, PULP_CBC_CMD, value

def new_pattern_problem(finish, length_s, cost_s, ap_upper_bound, demand_duals):
    prob = LpProblem("NewPatternProblem", LpMaximize)

    # Decision variables - Pattern
    ap = {f: LpVariable(f"ap_{f}", 0, ap_upper_bound[f], cat="Integer") for f in finish.keys()}

    # Objective function
    # maximize marginal_cost:
        #    sum{f in F} ap[f] * demand_dual[f] - c;
    prob += lpSum(ap[f] * demand_duals[f] for f in finish.keys()) - cost_s, "MarginalCost"

    # Constraints
    # subject to stock_length:
        #    sum{f in F} ap[f] * length_f[f] <= length_s;
    prob += lpSum(ap[f] * finish[f]["length"] for f in finish.keys()) <= length_s, "StockLength"

    # Solve the problem
    prob.solve(PULP_CBC_CMD(msg=False, options=['--solver', 'highs']))

    marg_cost = value(prob.objective)
    pattern = {f: int(ap[f].varValue) for f in finish.keys()}
    return marg_cost, pattern

def generate_pattern_dual(stocks, finish, patterns):
    prob = LpProblem("GeneratePatternDual", LpMinimize)

    # Sets
    F = list(finish.keys())
    P = list(range(len(patterns)))

    # Parameters
    s = {p: patterns[p]["stock"] for p in range(len(patterns))}
    c = {p: stocks[s[p]]["cost"] for p in range(len(patterns))}
    a = {(f, p): patterns[p]["cuts"][f] for p in P for f in F}
    demand_finish = {f: finish[f]["demand"] for f in F}

    # Decision variables
    # var x{P} >= 0; # relaxed integrality
    x = {p: LpVariable(f"x_{p}", 0, None, cat="Continuous") for p in P}

    # Objective function
    # minimize cost:
    #         sum{p in P} c[p] * x[p];
    prob += lpSum(c[p] * x[p] for p in P), "Cost"

    # Constraints
    # sum{p in P} a[f,p]*x[p] >= demand_finish[f];
    for f in F:
        prob += lpSum(a[f, p] * x[p] for p in P) >= demand_finish[f], f"Demand_{f}"

    # Solve the problem
    prob.solve(PULP_CBC_CMD(msg=False, options=['--solver', 'highs']))

    # Extract dual values
    dual_values = {f: prob.constraints[f"Demand_{f}"].pi for f in F}

    ap_upper_bound = {
        f: max([int(stocks[s]["length"] / finish[f]["length"]) for s in stocks.keys()])
        for f in F
    }
    demand_duals = {f: dual_values[f] for f in F}

    marginal_values = {}
    pattern = {}
    for s in stocks.keys():
        marginal_values[s], pattern[s] = new_pattern_problem(
            finish, stocks[s]["length"], stocks[s]["cost"], ap_upper_bound, demand_duals
        )

    s = max(marginal_values, key=marginal_values.get)
    new_pattern = {"stock": s, "cuts": pattern[s]}
    return new_pattern


In [7]:
stocks = {
    "A": {"length": 5, "cost": 6},
    "B": {"length": 6, "cost": 7},
    "C": {"length": 9, "cost": 10},
}

finish = {
    "S": {"length": 2, "demand": 20},
    "M": {"length": 3, "demand": 10},
    "L": {"length": 4, "demand": 20},
}

patterns = make_naive_patterns(stocks, finish)
generate_pattern_dual(stocks, finish, patterns)

{'stock': 'C', 'cuts': {'S': 1, 'M': 1, 'L': 1}}

In [8]:
# stocks = {
#     "log": {"length": 100, "cost": 1},
# }

# finish = {
#     1: {"length": 75.0, "demand": 38},
#     2: {"length": 75.0, "demand": 44},
#     3: {"length": 75.0, "demand": 30},
#     4: {"length": 75.0, "demand": 41},
#     5: {"length": 75.0, "demand": 36},
#     6: {"length": 53.8, "demand": 33},
#     7: {"length": 53.0, "demand": 36},
#     8: {"length": 51.0, "demand": 41},
#     9: {"length": 50.2, "demand": 35},
#     10: {"length": 32.2, "demand": 37},
#     11: {"length": 30.8, "demand": 44},
#     12: {"length": 29.8, "demand": 49},
#     13: {"length": 20.1, "demand": 37},
#     14: {"length": 16.2, "demand": 36},
#     15: {"length": 14.5, "demand": 42},
#     16: {"length": 11.0, "demand": 33},
#     17: {"length": 8.6, "demand": 47},
#     18: {"length": 8.2, "demand": 35},
#     19: {"length": 6.6, "demand": 49},
#     20: {"length": 5.1, "demand": 42},
# }

patterns = make_naive_patterns(stocks, finish)

print("Testing generate_patterns_dual: ", end="")
# %timeit 
new_pattern = generate_pattern_dual(stocks, finish, patterns)
while new_pattern not in patterns:
    patterns.append(new_pattern)
    new_pattern = generate_pattern_dual(stocks, finish, patterns)
    print(end=".")

    x, cost = cut_patterns_ortools(stocks, finish, patterns)
    print(f" Cost = {cost}")

Testing generate_patterns_dual: . Cost = 173.99999999999997
. Cost = 169.99999999999997
